In [1]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MaticF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path = "dataset.csv"
df = pd.read_csv(path)
print(df)

               id  sentiment      target  \
0      hasoc_en_1       none         NaN   
1      hasoc_en_2       hate    targeted   
2      hasoc_en_3       none         NaN   
3      hasoc_en_4       none         NaN   
4      hasoc_en_5       none         NaN   
...           ...        ...         ...   
12647        5642    hateful      origin   
12648        5643    hateful       other   
12649        5644    hateful       other   
12650        5645    hateful  disability   
12651        5646  offensive      origin   

                                                    text      source  
0      #DhoniKeepsTheGlove | WATCH: Sports Minister K...  25_twitter  
1      @politico No. We should remember very clearly ...  25_twitter  
2      @cricketworldcup Guess who would be the winner...  25_twitter  
3      Corbyn is too politically intellectual for #Bo...  25_twitter  
4      All the best to #TeamIndia for another swimmin...  25_twitter  
...                                          

In [3]:
corpus = df[["text","sentiment"]]
print(corpus)

                                                    text  sentiment
0      #DhoniKeepsTheGlove | WATCH: Sports Minister K...       none
1      @politico No. We should remember very clearly ...       hate
2      @cricketworldcup Guess who would be the winner...       none
3      Corbyn is too politically intellectual for #Bo...       none
4      All the best to #TeamIndia for another swimmin...       none
...                                                  ...        ...
12647  @user @user @user Maybe but @user who lied abo...    hateful
12648              @user @user You\u2019re a retard :) x    hateful
12649  (2/2) like no you fucking twat I'm doing damag...    hateful
12650            If your bf is retarded guess what u r 2    hateful
12651             @user whats the ching chong stuff mean  offensive

[12652 rows x 2 columns]


In [21]:
from nltk.stem.snowball import SnowballStemmer
import re

stemmer = SnowballStemmer("english")

corpus["tokenized"] = corpus['text'].astype(str).apply(nltk.word_tokenize)
prestemmed = corpus["tokenized"].tolist()
stemmed = []
unique = []
for tokenized_sentence in prestemmed:
    filt = []
    for word in tokenized_sentence:
        if re.search('[a-zA-Z]', word):
            filt.append(word)
    stems = [stemmer.stem(f) for f in filt]
    stemmed.append(stems)
    for s in stems:
        if s not in unique:
            unique.append(s)
corpus["stemmed"] = stemmed

<ipython-input-21-0214bbed3400>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus["tokenized"] = corpus['text'].astype(str).apply(nltk.word_tokenize)
<ipython-input-21-0214bbed3400>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus["stemmed"] = stemmed


In [57]:
sentiment = corpus["sentiment"].tolist()      
stemmed = corpus["stemmed"].tolist()
cfd = nltk.ConditionalFreqDist()
ht = ['hate', 'profanity', 'hateful', 'abusive', 'offensive', 'disrespectful', 'fearful']
for sent, stemmed_sentence in zip(sentiment,stemmed):
    for stem in stemmed_sentence:
        if sent in ht:
            cfd["hate"][stem] += 1
        else:
            cfd["none"][stem] += 1
print(cfd.keys())

dict_keys(['none', 'hate'])


In [64]:
meja = 3.84
pomembne = []
all_classes = list(cfd.keys())
print(all_classes)
for key in cfd.keys():
    print(f"Trenutni klass: {key}")
    other_classes = []
    for o in all_classes:
        if o != key:
            other_classes.append(o)
    S = cfd.N()
    R = cfd[key].N()
    for u in unique:
        chi = 0

        a = cfd[key][u] if u in cfd[key] else 0
        b = sum([cfd[o][u] if u in cfd[o] else 0 for o in other_classes])
        if a == 0 and b == 0:
            continue
        #Pricakovana vrednost
        E = (a+b)/(S+R)
        #Opazovana vrednost
        O = (a)/S

        chi += ((O-E)**2)/E
        print(chi)
        if chi > meja:
            print(u)
            pomembne.append((u,key))
        
print(pomembne)
    

['none', 'hate']
Trenutni klass: none
3.7630356033496464e-05
4.97024491299759e-05
1.3331302629475585e-06
2.4487056083200704e-06
5.022083758456412e-07
5.022083758456412e-07
6.592141890074853e-06
3.4123040766353833e-07
0.00018242711729557262
2.5186225790004195e-06
2.6573713394186435e-05
6.27823079854441e-05
2.9041487762495046e-07
7.010279264691457e-07
0.00012160654601969409
3.5326090470343634e-06
0.0017833868627647183
4.182512691278945e-05
0.0005233070939314559
0.002292986233739812
3.590008592683191e-06
0.000306633827715544
8.01949835142713e-06
0.0016978403036706788
6.796914623112423e-05
6.080441492527344e-06
0.0006891642984531785
0.0002901775475541818
0.0004155255518342755
1.3570492110526787e-06
0.0005340866464949418
4.882861388463991e-07
0.0017135149461127367
4.699194271878693e-06
2.7034279282142642e-05
5.022083758456412e-07
2.5794974299394675e-06
0.0002774493350671339
0.00021962143543236364
0.00011263287151474498
4.8601966010874236e-05
1.841484244291177e-05
5.49965780764738e-05
0.0013

7.422724619500586e-05
1.0044167516912824e-06
8.395408596668065e-07
5.43834069079181e-07
5.022083758456412e-07
5.851791773858844e-07
3.287751296308645e-06
1.3688581455782156e-06
5.022083758456412e-07
0.00015539948527409087
8.17634238870346e-07
7.572913708450678e-05
3.4902286641472404e-06
2.249173432511431e-05
2.943192872587448e-07
6.751660913460758e-07
2.1203893922698104e-08
1.8465814486031767e-06
5.208734773147058e-08
7.1237051508500386e-06
5.022083758456412e-07
4.6164536215079417e-07
1.2249156592882349e-05
2.943192872587448e-07
9.225528146413685e-06
5.851791773858844e-07
9.527232897924197e-07
4.400235204975374e-05
4.6164536215079417e-07
1.0044167516912824e-06
1.3867330899278751e-05
1.8465814486031767e-06
2.3082268107539705e-06
1.0029240269806797e-05
5.886385745174896e-07
1.3849360864523826e-06
6.751660913460758e-07
1.1716305591028566e-05
4.622443633092918e-06
5.5263752376223575e-06
5.022083758456412e-07
1.2722336353618863e-07
5.022083758456412e-07
2.3063820366034213e-06
2.787386518526

5.022083758456412e-07
2.7150077035499103e-07
1.0044167516912824e-06
3.5930571230348973e-06
5.851791773858844e-07
3.778578218043236e-07
5.851791773858844e-07
5.430015407099821e-07
4.227809757137707e-06
1.5706862022996887e-05
5.208734773147058e-08
6.751660913460758e-07
2.1203893922698104e-08
2.1203893922698104e-08
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
1.0044167516912824e-06
5.022083758456412e-07
5.022083758456412e-07
3.628011957722592e-05
2.5794974299394675e-06
5.022083758456412e-07
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
3.5635175991985796e-05
8.483175675242862e-06
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
2.787386518526365e-06
3.9683149015299946e-08
5.57477303705273e-06
4.227809757137707e-06
2.787386518526365e-06
2.537020240263764e-05
1.2897487149697338e-05
5.208734773147058e-08
4.622443633092918e-06
2.787386518526365e-06
1.6724319111158187e

1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
1.0044167516912824e-06
5.022083758456412e-07
1.5066251275369233e-06
2.1203893922698104e-08
5.022083758456412e-07
4.064189130394593e-06
2.5978885937230096e-05
1.3849360864523826e-06
2.9936304963597174e-05
5.208734773147058e-08
5.022083758456412e-07
7.1270351983971616e-06
4.6164536215079417e-07
2.3836439738464675e-08
9.225528146413685e-06
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
1.0044167516912824e-06
5.208734773147058e-08
5.208734773147058e-08
2.1203893922698104e-08
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
9.232907243015883e-07
9.232907243015883e-07
3.0132502550738465e-06
1.0044167516912824e-06
7.1237051508500386e-06
3.2158895902949605e-05
6.8614935379767866e-06
5.022083758456412e-07
5.022083758456412e-07
2.531613826249477e-05
9.694552605166674e-06
2.3063820

6.8614935379767866e-06
4.6164536215079417e-07
1.1465573862418915e-05
2.3063820366034213e-06
2.787386518526365e-06
3.5154586309194866e-06
2.3063820366034213e-06
2.3063820366034213e-06
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
1.0044167516912824e-06
5.022083758456412e-07
5.022083758456412e-07
9.232907243015883e-07
5.022083758456412e-07
1.0044167516912824e-06
1.833219269215794e-05
2.683906889777564e-07
5.022083758456412e-07
5.022083758456412e-07
1.5066251275369233e-06
9.232907243015883e-07
5.022083758456412e-07
5.022083758456412e-07
8.395408596668065e-07
1.0044167516912824e-06
5.022083758456412e-07
5.5263752376223575e-06
1.7773834688996625e-05
6.331834731414731e-06
3.125240863888235e-07
2.9603442340267997e-06
1.0044167516912824e-06
5.022083758456412e-07
5.022083758456412e-07
5.022083758456412e-07
1.0044167516912824e-06
1.0044167516912824e-06
5.022083758456412e-07
3.317461593624319e-06
1.9451783496217558e-05
4.6164536215079417e-07
5.022083758456412e-07
2.12038939226

...

8.347627227403661e-07
6.678101781922929e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
4.173813613701828e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
3.3390508909614644e-06
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.34

8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
3.3390508909614644e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.3476

8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
2.5042881682210974e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.34

8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
2.5042881682210974e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
1.6695254454807322e-06
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627

8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.347627227403661e-07
8.34762722

In [62]:
print(unique)

['dhonikeepstheglov', 'watch', 'sport', 'minist', 'kiren', 'rijiju', 'issu', 'statement', 'back', 'ms', 'dhoni', 'over', 'balidaan', 'badg', 'tell', 'bcci', 'to', 'take', 'up', 'the', 'matter', 'with', 'icc', 'and', 'keep', 'govern', 'in', 'know', 'as', 'nation', "'s", 'pride', 'is', 'involv', 'https', '//t.co/zuo5335rjr', 'politico', 'no', 'we', 'should', 'rememb', 'veri', 'clear', 'that', 'individual1', 'just', 'admit', 'treason', 'trumpisatraitor', 'mccainsahero', 'johnmccainday', 'cricketworldcup', 'guess', 'who', 'would', 'be', 'winner', 'of', 'this', 'cwc19', 'team', 'get', 'maximum', 'point', 'from', 'abandon', 'match', 'shameonicc', 'wivseng', 'corbyn', 'too', 'polit', 'intellectu', 'for', 'borisjohnsonshouldnotbepm', 'ca', "n't", 'wait', 'generalelectionnow', '//t.co/pt8kmjfxjj', 'all', 'best', 'teamindia', 'anoth', 'swim', 'competit', 'on', 'sunday', 'against', 'pakistan', 'indvpak', 'cwc19rain', '//t.co/mg2cie0zib', 'kellymiller513', 'therealoj32', 'i', 'hope', 'you', 'wipe'